If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
## If running in Colab
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# v8+ should be fine
!java -version

openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


In [55]:
import os
from subprocess import Popen, PIPE, STDOUT
import pandas as pd

splitter_jar_path = '../../SentenceSplitter.jar'
splitter_model_path = '../../es-sentence-splitter-model-spaccc.bin'
text_path = '../../multicardioner_train+dev_240429/track1/cardioccc_dev/brat'
annotations_path = '../../multicardioner_train+dev_240429/track1/cardioccc_dev/brat'
output_path = '../../multicardioner_train+dev_240429/track1/cardioccc_dev/sentences' 
sents_per_batch = 1 # max number of sentences in each output file

In [56]:
from pathlib import Path

for file_name in os.listdir(text_path):
    if '.txt' in file_name: # and 'casos_clinicos_cardiologia129' in file_name:
        file_name_no_ext = file_name.rstrip('.txt')
                 
        file_path = Path(f'{text_path}/{file_name}')
        full_text = file_path.read_text()
        
        p = Popen(['java', '-jar', splitter_jar_path , f'{text_path}/{file_name}', splitter_model_path], stdout=PIPE, stderr=STDOUT, text=True)

        print("Processing: ", file_name_no_ext)
        
        #T105	FARMACO 911 922	azatioprina

        annotations = pd.read_csv(f'{annotations_path}/{file_name_no_ext}.ann', sep='\t', 
                                  names=['ann_type', 'entity_type', 'text'], encoding='utf-8')

        # drop annotator comments
        annotations.drop(annotations[annotations['ann_type'].str.startswith('#')].index, inplace=True)

        annotations['start_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[1]))
        annotations['end_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[2]))
        annotations['entity_type'] = annotations['entity_type'].apply(lambda v: v.split()[0])
        annotations.sort_values(by=['start_pos'], inplace=True, ignore_index=True)

        total_chars = 0
        annotation_index = 0
        total_annotations = len(annotations)
        line_number = 0
        batch = []
        batch_number = 1
        for line in p.stdout:
            line_number += 1
            batch.append(line)

            if line_number % sents_per_batch == 0:
                batch_number += 1
                current_batch_annotation_indecies = []
                text = ''.join(batch)
                #max_index = total_chars + len(text) # - 1;
                #max_index = total_chars + full_text[total_chars:].find(text) + len(text) - 1
                batch_start = total_chars + full_text[total_chars:].find(text.strip()) #+ len(text) - 1
                batch_end = batch_start + len(text) - 1
                #print(text, batch_start, batch_end)
                if batch_start < 0:
                    print('batch_start', full_text[total_chars:], text, total_chars)
                
                while annotation_index < total_annotations and annotations.iloc[annotation_index]['start_pos'] >= batch_start and annotations.iloc[annotation_index]['start_pos'] <= batch_end:
                    annotation_text = str(annotations.iloc[annotation_index]['text'])
                    #print(annotation_text)
                    #new_start_pos = text.find(annotation_text)
                    new_start_pos = annotations.iloc[annotation_index]['start_pos'] - batch_start
                    
                    if new_start_pos < 0:
                        print('new_start_pos', text, annotation_text, batch_number, max_index)
                        
                    annotations.at[annotation_index, 'start_pos'] = new_start_pos
                    annotations.at[annotation_index, 'end_pos'] = new_start_pos + len(annotation_text)
                    current_batch_annotation_indecies.append(annotation_index)
                    annotation_index += 1

                with open(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{batch_number}.txt', 'w', encoding='utf-8') as out_file:
                    out_file.write(text)

                current_batch_annotations = annotations.iloc[current_batch_annotation_indecies,].copy().reset_index(drop=True)
                current_batch_annotations['entity_type_with_positions'] = None
                if not current_batch_annotations.empty:
                    current_batch_annotations['row_name'] = current_batch_annotations.apply(lambda row: row.name + 1, axis=1)
                    current_batch_annotations['ann_type'] = current_batch_annotations.apply(lambda row: f'{row["ann_type"].rstrip("0123456789")}{row.name + 1}', axis=1)
                    current_batch_annotations['entity_type_with_positions'] = current_batch_annotations.apply(lambda row: f'{row["entity_type"]} {row["start_pos"]} {row["end_pos"]}', axis=1)
                    current_batch_annotations.drop(columns=['start_pos', 'end_pos', 'entity_type'], inplace=True)

                current_batch_annotations.to_csv(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{batch_number}.ann', sep='\t', header=False, index=False, columns=['ann_type', 'entity_type_with_positions', 'text'], encoding='utf-8')

                total_chars = batch_end #len(text) # -1
                batch.clear()
                
        if annotation_index < total_annotations:
            print('missing annotations')
        #break

Processing:  casos_clinicos_cardiologia10
Processing:  casos_clinicos_cardiologia102
Processing:  casos_clinicos_cardiologia108
Processing:  casos_clinicos_cardiologia111
Processing:  casos_clinicos_cardiologia112
Processing:  casos_clinicos_cardiologia115
Processing:  casos_clinicos_cardiologia120
Processing:  casos_clinicos_cardiologia125
Processing:  casos_clinicos_cardiologia128
Processing:  casos_clinicos_cardiologia129
Processing:  casos_clinicos_cardiologia13
Processing:  casos_clinicos_cardiologia132
Processing:  casos_clinicos_cardiologia134
Processing:  casos_clinicos_cardiologia135
Processing:  casos_clinicos_cardiologia136
Processing:  casos_clinicos_cardiologia138
Processing:  casos_clinicos_cardiologia139
Processing:  casos_clinicos_cardiologia140
Processing:  casos_clinicos_cardiologia141
Processing:  casos_clinicos_cardiologia143
Processing:  casos_clinicos_cardiologia144
Processing:  casos_clinicos_cardiologia146
Processing:  casos_clinicos_cardiologia148
Processing:  

Processing:  casos_clinicos_cardiologia499
Processing:  casos_clinicos_cardiologia50
Processing:  casos_clinicos_cardiologia500
Processing:  casos_clinicos_cardiologia502
Processing:  casos_clinicos_cardiologia505
Processing:  casos_clinicos_cardiologia509
Processing:  casos_clinicos_cardiologia51
Processing:  casos_clinicos_cardiologia511
Processing:  casos_clinicos_cardiologia515
Processing:  casos_clinicos_cardiologia519
Processing:  casos_clinicos_cardiologia520
Processing:  casos_clinicos_cardiologia526
Processing:  casos_clinicos_cardiologia531
Processing:  casos_clinicos_cardiologia532
Processing:  casos_clinicos_cardiologia533
Processing:  casos_clinicos_cardiologia534
Processing:  casos_clinicos_cardiologia538
Processing:  casos_clinicos_cardiologia542
Processing:  casos_clinicos_cardiologia543
Processing:  casos_clinicos_cardiologia546
Processing:  casos_clinicos_cardiologia548
Processing:  casos_clinicos_cardiologia549
Processing:  casos_clinicos_cardiologia550
Processing:  